In [13]:
#Xuất ra top 10 bộ phim (chưa được người dùng xem) có dự đoán rating cao nhất cho UserID =20
import pandas as pd

ratings = pd.read_csv("/content/drive/MyDrive/Giảng dạy/Hệ khuyến nghị/movielens/ratings.csv")
movies = pd.read_csv("/content/drive/MyDrive/Giảng dạy/Hệ khuyến nghị/movielens/movies.csv")

In [21]:
#lấy ra rating của một user cho 1 bộ phim
def get_rating(userid,movieid):
    return (ratings.loc[(ratings.userId==userid) & (ratings.movieId == movieid),'rating'].iloc[0])

get_rating(8,2)

4.0

In [26]:
from numpy import sqrt

def pearson_correlation_score(user1, user2):
  both_watch_count = [] # danh sách các bộ phim được xem chung bởi user1 và user2
  list_movie_user1 = ratings.loc[ratings.userId == user1, 'movieId'].to_list()
  list_movie_user2 = ratings.loc[ratings.userId == user2, 'movieId'].to_list()

  for element in list_movie_user1:
    if element in list_movie_user2:
       both_watch_count.append(element)
  
  if(len(both_watch_count) == 0):
    return 0;
  
  rating_sum_1 = sum([get_rating(user1, i) for i in both_watch_count])
  avg_rating_sum_1 = rating_sum_1/len(both_watch_count) # rating trung bình user1

  rating_sum_2 = sum([get_rating(user2, i) for i in both_watch_count])
  avg_rating_sum_2 = rating_sum_2/len(both_watch_count)# rating trung bình user2

  tu = sum([(get_rating(user1, i) - avg_rating_sum_1)*(get_rating(user2, i) - avg_rating_sum_2)  for i in both_watch_count])
  
  mau = sqrt(sum([pow((get_rating(user1, i) - avg_rating_sum_1),2) for i in both_watch_count]))*sqrt(sum([pow((get_rating(user2, i) - avg_rating_sum_2),2) for i in both_watch_count]))

  if(mau ==0 ):
    return 0
  
  return tu/mau


In [29]:
pearson_correlation_score(10,12)

0.5625000000000001

In [36]:
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity

def distance_similarity_score(user1,user2):

    both_watch_count = 0
    for element in ratings.loc[ratings.userId==user1,'movieId'].tolist():
        if element in ratings.loc[ratings.userId==user2,'movieId'].tolist():
            both_watch_count += 1
    if both_watch_count == 0 :
        return 0
    

    rating1 = []
    rating2 = []
    for element in ratings.loc[ratings.userId==user1,'movieId'].tolist():
        if element in ratings.loc[ratings.userId==user2,'movieId'].tolist():
            rating1.append(get_rating(user1,element))
            rating2.append(get_rating(user2,element))

    print(rating1)
    print(rating2)

    return dot(rating1, rating2)/(norm(rating1)*norm(rating2))

print('Distance based similarity between user ids X & u1: {}'.format(distance_similarity_score(1,2)))
print('Distance based similarity between user ids X & u2: {}'.format(distance_similarity_score(1,3)))
print('Distance based similarity between user ids X & u3: {}'.format(distance_similarity_score(1,4)))

[5.0, 5.0]
[4.0, 4.0]
Distance based similarity between user ids X & u1: 0.9999999999999998
[5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0]
[0.5, 3.5, 4.5, 0.5, 0.5, 2.0, 5.0]
Distance based similarity between user ids X & u2: 0.7919033104178548
[5.0, 4.0, 5.0, 3.0, 4.0, 5.0, 5.0, 4.0, 5.0, 3.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 2.0, 4.0, 5.0, 5.0, 4.0, 5.0, 3.0, 5.0, 5.0, 4.0, 5.0, 4.0, 5.0, 4.0, 5.0, 4.0, 5.0, 5.0, 5.0, 4.0, 5.0, 1.0, 5.0, 4.0, 4.0]
[2.0, 2.0, 5.0, 1.0, 1.0, 5.0, 2.0, 5.0, 5.0, 3.0, 5.0, 4.0, 2.0, 4.0, 5.0, 5.0, 5.0, 5.0, 3.0, 4.0, 4.0, 4.0, 5.0, 4.0, 4.0, 4.0, 3.0, 2.0, 4.0, 5.0, 5.0, 5.0, 3.0, 4.0, 1.0, 1.0, 5.0, 5.0, 2.0, 4.0, 4.0, 4.0, 4.0, 1.0, 3.0]
Distance based similarity between user ids X & u3: 0.9328095891279707


13

In [92]:
def most_similar_user(user1, number_of_user, similarity_name):

  user_ID = ratings.userId.unique().tolist()
  print(len(user_ID))

  if(similarity_name == "pearson"):
    similarity_score = [(pearson_correlation_score(user1, user_i),user_i)  for user_i in user_ID[0:300] if user_i != user1] #danh sách user quá nhiều nên tình chỉ tính tên dánh sách có 50 users

  # if(similarity_name == "cosine"):

  

  similarity_score.sort() #tăng dần
  similarity_score.reverse() #tăng dần

  return similarity_score[:10] # có thể thay đổi số lượng lân cận


In [93]:
list_user_popular = most_similar_user(20, 10, "pearson")

610


In [94]:
print(list_user_popular)

[(1.0000000000000002, 209), (1.0000000000000002, 127), (1.0000000000000002, 54), (1.0, 149), (1.0, 56), (0.9999999999999999, 296), (0.9999999999999998, 183), (0.9999999999999998, 175), (0.9999999999999998, 157), (0.9819805060619659, 191)]


In [95]:
#danh sách các bộ phim đã rating bởi người dùng
def get_movieids(userid):
    return (ratings.loc[(ratings.userId==userid),'movieId'].tolist())

#lất ra tên của một bộ phim
def get_movie_title(movieid):
    return (movies.loc[(movies.movieId == movieid),'title'].iloc[0])

In [108]:
total1 = {}

total1[3] = 0
total1[3] = 9

total1[3]

9

In [118]:
#lấy ra danh sách khuyến nghị từ top populars
def get_recommendation1_(userid):
    # user_ids = ratings.userId.unique().tolist()
    total = {}
    similariy_sum = {}
    
    # Iterating over subset of user ids.
    for pearson, user in list_user_popular:
        
        score = pearson

        for movieid in get_movieids_(user): #-> dánh sách các id movie đã xem bởi user khác và khởi tạo giá trị =0
          if movieid not in get_movieids_(userid):
            total[movieid] = 0
            similariy_sum[movieid] = 0

        for movieid in get_movieids_(user): #-> dánh sách các id movie đã xem bởi user khác
          if movieid not in get_movieids_(userid):
            total[movieid] += get_rating(user,movieid) * score
            similariy_sum[movieid] += score

        # for movieid in get_movieids_(user): #-> dánh sách các id movie đã xem vởi user khác
        #     # Only considering not watched/rated movies
        #     if movieid not in get_movieids(userid):# or get_rating(userid,movieid) == 0:
        #       total[movieid] = 0
        #       total[movieid] += get_rating(user,movieid) * score #=> person nhân cho ratings của user ứng cử tương ứng
        #     similariy_sum[movieid] = 0
        #     similariy_sum[movieid] += score # tổng tổng độ tương tự của user target và các user khác 

    
    # Normalizing ratings
    ranking = [(tot/similariy_sum[movieid],movieid) for movieid,tot in total.items()]
    ranking.sort() # sắp xếp tăng dần
    ranking.reverse() # đẩo chiều cho giảm dần
    
    
    recommendations = [(get_movie_title_(movieid), score) for score,movieid in ranking]
    return recommendations[:10]
print(get_recommendation1_(20))

[('Inception (2010)', 5.0), ('Inglourious Basterds (2009)', 5.0), ('Ponyo (Gake no ue no Ponyo) (2008)', 5.0), ('Watchmen (2009)', 5.0), ('Michael Clayton (2007)', 5.0), ('Step Up (2006)', 5.0), ('Take the Lead (2006)', 5.0), ('Goal! The Dream Begins (Goal!) (2005)', 5.0), ('Ice Princess (2005)', 5.0), ('Memories of Murder (Salinui chueok) (2003)', 5.0)]
